# Bucketing

# 1 Load Data

In [1]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

In [2]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

Use a single year to keep execution plans small

In [3]:
raw_weather = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))

In [4]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

## 1.2 Load Station Metadata

In [5]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# 2 Bucketing

In [24]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.sources.bucketing.enabled", True)

## 2.1 Normal Join

In [7]:
result = weather.join(stations, (weather["usaf"] == stations["usaf"]) & (weather["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(5) SortMergeJoin [usaf#87, wban#88], [usaf#122, wban#123], Inner
:- *(2) Sort [usaf#87 ASC NULLS FIRST, wban#88 ASC NULLS FIRST], false, 0
:  +- Exchange hashpartitioning(usaf#87, wban#88, 200)
:     +- *(1) Project [2003 AS year#84, substring(value#82, 5, 6) AS usaf#87, substring(value#82, 11, 5) AS wban#88, substring(value#82, 16, 8) AS date#89, substring(value#82, 24, 4) AS time#90, substring(value#82, 42, 5) AS report_type#91, substring(value#82, 61, 3) AS wind_direction#92, substring(value#82, 64, 1) AS wind_direction_qual#93, substring(value#82, 65, 1) AS wind_observation#94, (cast(cast(substring(value#82, 66, 4) as float) as double) / 10.0) AS wind_speed#95, substring(value#82, 70, 1) AS wind_speed_qual#96, (cast(cast(substring(value#82, 88, 5) as float) as double) / 10.0) AS air_temperature#97, substring(value#82, 93, 1) AS air_temperature_qual#98]
:        +- *(1) Filter (isnotnull(substring(value#82, 5, 6)) && isnotnull(substring(value#82, 11, 5)))
:    

## 2.2 Bucketed Join

In [21]:
weather.write \
    .bucketBy(200, "usaf", "wban") \
    .sortBy("usaf", "wban") \
    .mode("overwrite") \
    .saveAsTable("weather_buckets")

Inspect table in Hive

In [27]:
spark.sql("DESCRIBE EXTENDED weather_buckets").show(100)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                year|                 int|   null|
|                usaf|              string|   null|
|                wban|              string|   null|
|                date|              string|   null|
|                time|              string|   null|
|         report_type|              string|   null|
|      wind_direction|              string|   null|
| wind_direction_qual|              string|   null|
|    wind_observation|              string|   null|
|          wind_speed|              double|   null|
|     wind_speed_qual|              string|   null|
|     air_temperature|              double|   null|
|air_temperature_qual|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|             default|       |
|           

Try join again, with bucketed table

In [25]:
weather_hive = spark.read.table("weather_buckets")
result = weather_hive.join(stations, (weather_hive["usaf"] == stations["usaf"]) & (weather_hive["wban"] == stations["wban"]))
result.explain()

== Physical Plan ==
*(4) SortMergeJoin [usaf#694, wban#695], [usaf#122, wban#123], Inner
:- *(1) Sort [usaf#694 ASC NULLS FIRST, wban#695 ASC NULLS FIRST], false, 0
:  +- *(1) Project [year#693, usaf#694, wban#695, date#696, time#697, report_type#698, wind_direction#699, wind_direction_qual#700, wind_observation#701, wind_speed#702, wind_speed_qual#703, air_temperature#704, air_temperature_qual#705]
:     +- *(1) Filter (isnotnull(wban#695) && isnotnull(usaf#694))
:        +- *(1) FileScan parquet default.weather_buckets[year#693,usaf#694,wban#695,date#696,time#697,report_type#698,wind_direction#699,wind_direction_qual#700,wind_observation#701,wind_speed#702,wind_speed_qual#703,air_temperature#704,air_temperature_qual#705] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://ip-10-200-101-220.eu-central-1.compute.internal:8020/user/hive/warehouse..., PartitionFilters: [], PushedFilters: [IsNotNull(wban), IsNotNull(usaf)], ReadSchema: struct<year:int,usaf:string,wban:strin

The following attributes have to match
* bucketing columns
* number of buckets = number of partitions